#### Retrieval:

In [2]:
from langchain.embeddings.base import Embeddings
import numpy as np

class SentenceTransformerEmbeddings(Embeddings): #We need to inherit from Embeddings for integrating into langchain pipeline
    def __init__(self, model):
        self.model = model

    def embed_documents(self,texts):
        return [self.model.encode(text) for text in texts]

    def embed_query(self, query):
        return self.model.encode(query)

In [3]:
from sentence_transformers import SentenceTransformer

embedding_model = SentenceTransformer('all-mpnet-base-v2')

/opt/conda/lib/python3.11/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

/opt/conda/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [4]:
sentence_embeddings = SentenceTransformerEmbeddings(embedding_model)

In [10]:

from langchain_community.vectorstores import FAISS

db = FAISS.load_local("faiss_index", sentence_embeddings, allow_dangerous_deserialization=True)


In [11]:
# Perform a query by using the retriever
retriever = db.as_retriever()  # Convert FAISS into a retriever object
query = "What is phylosophy?"
retrieved_docs = retriever.get_relevant_documents(query)

# Print out the retrieved documents
for doc in retrieved_docs:
    print(doc.page_content)
    print('\n')


/tmp/ipykernel_414/2456047785.py:4: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use invoke instead.
  retrieved_docs = retriever.get_relevant_documents(query)


systematics, the field of biology that studies the diversity of life and the relationships of living things through time. Today, systematists typically treat “relatedness” solely in terms of recency of common ancestry, but this was not always the case. Pre-Darwinian taxonomists discussed the relationships of various groups and their place in the “natural system”, and while the rise of evolutionary theory allowed that one sense of relatedness was genealogical, it did not eliminate the idea of the broader notion. Debates about the role of phylogeny in classification and taxonomy were widespread (e.g., Huxley [ed.] 1940; Winsor 1995) though they began to take on a new form beginning in the late 1950s as collaborations turned into organized research programs pushing their agendas.  In his analysis of the period, David Hull (1988: ch. 5) titled one of his chapters “Systematists at War” and thus the name “The Systematics Wars” is sometimes used to describe the debates of the period. Hull (19

#### Question Answering:

In [12]:
from transformers import AutoModelForCausalLM, AutoTokenizer
device = "cuda" # the device to load the model onto

model = AutoModelForCausalLM.from_pretrained(
    "Qwen/CodeQwen1.5-7B-Chat",
    torch_dtype="auto",
    device_map="auto"
)


config.json:   0%|          | 0.00/701 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/31.7k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/3.89G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/3.95G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/3.95G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/2.71G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/212 [00:00<?, ?B/s]

In [13]:
tokenizer = AutoTokenizer.from_pretrained("Qwen/CodeQwen1.5-7B-Chat")

prompt = "how to enjoy life"
messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": prompt}
]

tokenizer_config.json:   0%|          | 0.00/972 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/4.46M [00:00<?, ?B/s]

In [14]:
from langchain_core.prompts import ChatPromptTemplate

template = ChatPromptTemplate([
     ('system', "You are a helpful assistant."),
    ('user', "{input}")
])

prompt  = template.invoke({'input': 'how to enjoy life?'})

In [15]:
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)
model_inputs = tokenizer([text], return_tensors="pt").to('cuda')

generated_ids = model.generate(
    model_inputs.input_ids,
    max_new_tokens=512
)
generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

In [16]:
response

"Enjoying life can vary for everyone, but here are some suggestions:\n\n1. Find your passion: Discover what makes you happy and what you love doing. This could be anything - painting, cooking, playing music, reading, or spending time with loved ones.\n\n2. Set achievable goals: Create a plan for your life that is achievable and realistic. Don't try to do everything at once; break it down into smaller, manageable tasks.\n\n3. Spend time with loved ones: Make time for family and friends, or join a community group to connect with like-minded individuals. Spending time with loved ones can boost your mood and create lasting memories.\n\n4. Travel and explore: Pursue new experiences, whether that means traveling to new places or trying new foods.\n\n5. Practice self-care: Take care of your physical, mental, and emotional well-being by eating a healthy diet, getting enough sleep, exercising regularly, and seeking support when you need it.\n\n6. Give back to your community: Volunteer or donate

#### Add retrieval to the question answering:

In [17]:
from langchain_core.prompts import ChatPromptTemplate 
# chat prompt template is used for running model in conversational format


system_prompt = (
    "You are an assistant for question-answering tasks. "
    "You're name is Dr.Phil"
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate([
    ('system', system_prompt),
    ('human', '{input}')])

In [31]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from langchain_core.runnables import Runnable
import torch



class CustomLLMRunnable(Runnable):
    def invoke(self, messages, **kwargs):
        text = tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True
        )
        
        model_inputs = tokenizer([text], return_tensors="pt").to('cuda')
        
        generated_ids = model.generate(
            model_inputs.input_ids,
            max_new_tokens=512
        )
        
        # Slice generated tokens to exclude input
        generated_ids = [
            output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
        ]
        
        # Decode the response and return it
        response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
        return response

custom_llm_runnable = CustomLLMRunnable()

messages = [
    {"role": "system", "content": "You are Dr. Phil."},
    {"role": "user", "content": "What is philosophy?"}
]

# Call the custom runnable
response = custom_llm_runnable.invoke(messages)

# Output the generated response
print(response)




Philosophy is the systematic exploration and analysis of the fundamental questions, problems, and propositions that govern human existence and the way in which the world is organized. 

Philosophy can be traced back to ancient Greek and Roman philosophers, who formulated various philosophical systems and concepts to gain a deeper understanding of reality, knowledge, and the nature of reality. 

Today, philosophy is a multidisciplinary field that draws from a wide range of disciplines, such as history, economics, science, and religion, to explore fundamental questions and problems. It aims to provide a more comprehensive understanding of the world and to challenge our assumptions about the nature of reality and the way in which we interact with it.

In summary, philosophy is the study of the fundamental questions, principles, and values that guide human inquiry and understanding. It is a broad and complex field that is relevant to a wide range of aspects of life.


In [36]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain

question_answer_chain = create_stuff_documents_chain(custom_llm_runnable, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [39]:
rag_chain.invoke({'input':'what is phylosophy?'})

TypeError: CustomLLMRunnable.invoke() takes 2 positional arguments but 3 were given

In [41]:
system_prompt = (
    "You are an assistant for question-answering tasks. "
    "You're name is Dr.Phil"
)

prompt = ChatPromptTemplate([
    ('system', system_prompt),
    ('human', 'what is phylosophy?')])

# Call the custom runnable
response = custom_llm_runnable.invoke(prompt)

# Output the generated response
print(response)


UndefinedError: 'langchain_core.prompts.chat.SystemMessagePromptTemplate object' has no attribute 'role'